In [6]:
from sklearn import datasets
import sklearn
import pandas as pd

In [8]:
X,y = sklearn.datasets.fetch_covtype(return_X_y=True)

In [9]:
file_path = 'output/MLP/Adult/predictions/p_1.npy'
with open(file_path, 'r') as file:
    content = file.read()
print(content)


array([5, 5, 2, ..., 3, 3, 3])